---
# Testing processing system
---

In [30]:
import os.path
import numpy as np
import pandas as pd
import time
from datetime import datetime

import shtReader_py.shtRipper as shtRipper
from source.Files_operating import read_dataFile, read_sht_data
from source.NN_environment import process_fragments, get_borders, normalise_series, down_to_zero
# from source.NN_environment import get_prediction_multi_unet

In [31]:
import tensorflow as tf
from source.NN_environment import unet_multi_model, dice_bce_loss

# TO DO: predict all fragment in one call
def get_prediction_multi_unet(data: np.array, POINTS_DIM=1024, NUM_CLASSES=2, ckpt_v=0) -> np.array:
    checkpoint_filepath = f'models/ckpt/checkpoint_{ckpt_v}_multi.weights.h5'

    precision = tf.keras.metrics.Precision()
    recall = tf.keras.metrics.Recall()
    
    model = unet_multi_model(POINTS_DIM, NUM_CLASSES)
    model.compile(optimizer='adam', loss=dice_bce_loss,
                  metrics=['acc', precision, recall])
    model.load_weights(checkpoint_filepath)
    
    l_edge = 0
    step = 256
    
    prediction_result = np.zeros((NUM_CLASSES, data.shape[0]))
    # print(prediction_result.shape, prediction_result)
    
    while l_edge + POINTS_DIM < data.shape[0]:
        predictions = model.predict(np.array([normalise_series(data[l_edge:l_edge + POINTS_DIM])]), verbose=0)
        # print(predictions.shape)
        for i in range(0, POINTS_DIM):
            prediction_result[0, l_edge + i] = predictions[0, i, 0]
            prediction_result[1, l_edge + i] = predictions[0, i, 1]
        l_edge += step
    
    if l_edge + POINTS_DIM - step != data.shape[0] - 1:
        predictions = model.predict(np.array([normalise_series(data[data.shape[0] - POINTS_DIM:])]), verbose=0)
        for i in range(0, POINTS_DIM):
            prediction_result[0, data.shape[0] - POINTS_DIM + i] = predictions[0, i, 0]
            prediction_result[1, data.shape[0] - POINTS_DIM + i] = predictions[0, i, 1]

    return prediction_result

In [39]:
filename = "sht44350"
filepath = "C:/Users/f.belous/Work/Projects/D-alpha-instability-search/data/sht/"
ckpt_v = 1

F_ID = filename[-5:]

In [40]:
df = read_sht_data(filename, filepath, data_name="D-alfa  хорда R=50 cm")
start_time = time.time()

In [41]:
predictions = get_prediction_multi_unet(df.ch1.to_numpy(), ckpt_v=ckpt_v)

C:\Users\f.belous\AppData\Local\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 52 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [42]:
df["unsync_ai_marked"] = predictions[0, :]
df["sync_ai_marked"] = predictions[1, :]

df["unsync_marked"] = down_to_zero(np.array(df["unsync_ai_marked"]), edge=0.5)
df["unsync_marked"] = process_fragments(np.array(df["ch1"]), np.array(df["unsync_marked"]), length_edge=30, scale=1.5)  # old version: length_edge=20, , scale=0

df["sync_marked"] = down_to_zero(np.array(df["sync_ai_marked"]), edge=0.5)
df["sync_marked"] = process_fragments(np.array(df["ch1"]), np.array(df["sync_marked"]), length_edge=30, scale=0)  # old version: length_edge=30, , scale=1.5

sxr_df = read_sht_data(filename, filepath, data_name="SXR 50 mkm")

comment = {"ai_marking": f'Processed NN prediction of ELMs (v{ckpt_v} multiclass model; trn-on: #44[184|194] )',
           "sync_proc_marks": f'Sync ELMs marks (by proc-sys v2.1-0scl; {datetime.now().strftime("%d.%m.%Y")})',
           "unsync_proc_marks": f'Unsync ELMs marks (by proc-sys v2.2-1.5scl; {datetime.now().strftime("%d.%m.%Y")})'}

to_pack = {
    "D-alpha, chord=50 cm": {
        'comment': f'SHOT: #{F_ID}',
        'unit': 'U(V)',
        # 'x': df.t,
        'tMin': df.t.min(),  # minimum time
        'tMax': df.t.max(),  # maximum time
        'offset': 0.0,  # ADC zero level offset
        'yRes': 0.001,  # ADC resolution: 0.0001 Volt per adc bit
        'y': df.ch1.to_list()
    },
    "SXR 50 mkm": {
        'comment': f'ADC #4, CH #6, SHOT: #{F_ID}',
        'unit': 'U(V)',
        # 'x': df.t,
        'tMin': df.t.min(),  # minimum time
        'tMax': df.t.max(),  # maximum time
        'offset': 0.0,  # ADC zero level offset
        'yRes': 0.001,  # ADC resolution: 0.0001 Volt per adc bit
        'y': sxr_df.ch1.to_list()
    },
    "Unsync ELM AI prediction": {
        'comment': comment["ai_marking"],
        'unit': 'U(V)',
        # 'x': df.t,
        'tMin': df.t.min(),  # minimum time
        'tMax': df.t.max(),  # maximum time
        'offset': 0.0,  # ADC zero level offset
        'yRes': 0.001,  # ADC resolution: 0.0001 Volt per adc bit
        'y': df.unsync_ai_marked.to_list()
    },
    "Unsync ELM mark": {
        'comment': comment["unsync_proc_marks"],
        'unit': 'U(V)',
        # 'x': df.t,
        'tMin': df.t.min(),  # minimum time
        'tMax': df.t.max(),  # maximum time
        'offset': 0.0,  # ADC zero level offset
        'yRes': 0.001,  # ADC resolution: 0.0001 Volt per adc bit
        'y': df.unsync_marked.to_list()
    },
    "Sync ELM AI prediction": {
        'comment': comment["ai_marking"],
        'unit': 'U(V)',
        # 'x': df.t,
        'tMin': df.t.min(),  # minimum time
        'tMax': df.t.max(),  # maximum time
        'offset': 0.0,  # ADC zero level offset
        'yRes': 0.001,  # ADC resolution: 0.0001 Volt per adc bit
        'y': df.sync_ai_marked.to_list()
    },
    "Sync ELM mark": {
        'comment': comment["sync_proc_marks"],
        'unit': 'U(V)',
        # 'x': df.t,
        'tMin': df.t.min(),  # minimum time
        'tMax': df.t.max(),  # maximum time
        'offset': 0.0,  # ADC zero level offset
        'yRes': 0.001,  # ADC resolution: 0.0001 Volt per adc bit
        'y': df.sync_marked.to_list()
    }
}

packed = shtRipper.ripper.write(path=filepath + "marked/", filename=f'{F_ID}_data.SHT', data=to_pack)

print(f"Result saved successfully to {filepath}marked/{F_ID}_data.SHT")
print(f"Took - {round(time.time() - start_time, 2)} s")

df.to_csv(filepath + f"marked/df/{F_ID}_data.csv", index=False)

Result saved successfully to C:/Users/f.belous/Work/Projects/D-alpha-instability-search/data/sht/marked/44350_data.SHT
Took - 106.0 s
